<a href="https://colab.research.google.com/github/buzzbing/service-recommendation/blob/main/ServiceRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:


import numpy as np
import pandas as pd
import sklearn
import collections
from sklearn.model_selection import train_test_split
# from mpl_toolkits.mplot3d import Axes3D
# from IPython import display
# from matplotlib import pyplot as plt

# import sklearn.manifold
# import tensorflow.compat.v1 as tf

In [131]:
dataset = pd.read_csv('/content/DatasetForMP.csv')

In [132]:
dataset.head()
dataset = dataset.sample(frac=1)
dataset.head()

,uid,lat,lng,serviceIDs,orderATime,orderId,itemID,title
98,QkbhObpo8fRvlCPtXiJIwZGhoKQ2,27.712660,85.294273,1677414535356,01-Mar-23,1678711894471,1677414535356,Antivirus
22,RmXmnJJ4Zpdyc2oSXYyniZg4VWD2,27.690675,85.310215,1677412835112,03-Feb-23,1678649288238,1677412835112,Thread & wax
11,4vlBpikHTKMLNmThxAQRcCpso152,27.678510,85.304612,1677412703069,19-Jan-23,1678724533927,1677412703069,Pedicure
17,5DSxQzlXn0ZBVWNTY04mR5RDJKf2,27.712632,85.294196,1678723452253,25-Jan-23,1678723877755,1678723452253,TV
16,5DSxQzlXn0ZBVWNTY04mR5RDJKf2,27.712632,85.294196,1678723595979,23-Jan-23,1678723862909,1678723595979,Refrigerator


In [133]:
dataset['uid'].nunique()

40

In [157]:
y = dataset['title']

In [158]:
y = pd.DataFrame(y)

In [159]:
print(type(y))

<class 'pandas.core.frame.DataFrame'>


In [160]:
X = dataset.drop(columns=['title','serviceIDs','itemID','uid','orderId'])

In [161]:
X.head()

,lat,lng,orderATime
98,27.712660,85.294273,01-Mar-23
22,27.690675,85.310215,03-Feb-23
11,27.678510,85.304612,19-Jan-23
17,27.712632,85.294196,25-Jan-23
16,27.712632,85.294196,23-Jan-23


In [162]:
X_train,X_valid,y_train,y_valid = train_test_split(X,y,train_size = 0.9,test_size = 0.1,random_state =10)

In [163]:
print(X_train.columns,end='\n\n')
object_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
print(object_cols)

Index(['lat', 'lng', 'orderATime'], dtype='object')

['orderATime']


In [164]:
object_nunique = list(map(lambda col: X_train[col].nunique(),object_cols))
d = dict(zip(object_cols,object_nunique))
sorted(d.items(),key=lambda x:x[1])

[('orderATime', 22)]

In [165]:
train_label_cols = [col for col in object_cols if set(X_valid[col]).issubset(set(X_train[col]))]
missing_label_cols = list(set(object_cols)-set(train_label_cols))
print(missing_label_cols)


[]


In [166]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(handle_unknown = 'use_encoded_value',unknown_value = -1)

label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

label_X_train[object_cols] = encoder.fit_transform(label_X_train[object_cols])
label_X_valid[object_cols] = encoder.transform(label_X_valid[object_cols])

label_X_train.head()


,lat,lng,orderATime
11,27.678510,85.304612,12.0
22,27.690675,85.310215,2.0
10,27.678510,85.304612,12.0
64,27.712665,85.294280,15.0
51,27.688382,85.290081,11.0


In [167]:
y.head()

,title
98,Antivirus
22,Thread & wax
11,Pedicure
17,TV
16,Refrigerator


In [168]:

_cols = [col for col in y_train.columns if y_train[col].dtype == 'object']
print(_cols)

['title']


In [169]:
encoder = OrdinalEncoder(handle_unknown = 'use_encoded_value',unknown_value = -1)

label_y_train = y_train.copy()
label_y_valid = y_valid.copy()

label_y_train[_cols] = encoder.fit_transform(label_y_train[_cols])
label_y_valid[_cols] = encoder.transform(label_y_valid[_cols])


print(label_y_train)


    title
11    8.0
22   14.0
10   14.0
64    4.0
51    3.0
..    ...
97   13.0
31   13.0
72    9.0
75    1.0
0    13.0

[90 rows x 1 columns]


In [170]:
label_y_train.head()

,title
11,8.0
22,14.0
10,14.0
64,4.0
51,3.0


In [171]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

model = RandomForestClassifier(n_estimators = 200, random_state = 10)
model.fit(label_X_train,label_y_train.title)
preds = model.predict(label_X_valid)




In [172]:
print(mean_absolute_error(label_y_valid.title,preds))

6.6


In [173]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(label_y_valid,preds )
print(accuracy)


0.1


In [174]:
label_X_train.head()

,lat,lng,orderATime
11,27.678510,85.304612,12.0
22,27.690675,85.310215,2.0
10,27.678510,85.304612,12.0
64,27.712665,85.294280,15.0
51,27.688382,85.290081,11.0


In [192]:
# ordertime = encoder.inverse_transform(label_X_train['orderATime'])

In [189]:
x = encoder.inverse_transform([
    model.predict(
        [[27.700652	,85.191222	,12.0]]
        )])

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [191]:
print(x)

[['Facial']]
